In [1]:
# !pip install pinecone-client
# !pip install langchain-pinecone


In [3]:
from pinecone import Pinecone, PodSpec
from dotenv import load_dotenv
import os
import json
import oci
from os import getenv
from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    OnDemandServingMode,
    EmbedTextDetails,
)


In [4]:
load_dotenv()

# Pinecone Environment
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
index_name = "sample-index"
namespace = "sample-namespace"
# Pinecone Client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Generative AI Environment
GEN_AI_INFERENCE_ENDPOINT = getenv('GEN_AI_INFERENCE_ENDPOINT')
COMPARTMENT_ID = getenv('COMPARTMENT_ID')
EMBED_MODEL_OCID = getenv('EMBED_MODEL_OCID')
CONFIG_PROFILE = "CHICAGO"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)
# Generative AI Client
genai = GenerativeAiInferenceClient(
    config=config,
    # signer=InstancePrincipalsSecurityTokenSigner(),
    service_endpoint=GEN_AI_INFERENCE_ENDPOINT,
    retry_strategy=oci.retry.NoneRetryStrategy(), 
    timeout=(10,240)
)


In [5]:
# Create index
pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine",
    spec=PodSpec(
        environment="gcp-starter"
    )
)


In [6]:
index = pc.Index(index_name)
index.describe_index_stats()


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [7]:
# Embed
def embed_text(query: list[str]):
    response = genai.embed_text(
        embed_text_details=EmbedTextDetails(
            serving_mode=OnDemandServingMode(
                model_id="cohere.embed-multilingual-v3.0"
            ),
            inputs=query,
            truncate="NONE",
            compartment_id=COMPARTMENT_ID,
            is_echo=True
        )
    )
    if not response.status == 200:
        raise Exception("Failed to obtain query response.")
    return response


In [8]:
file_path = './sample_data.json'
with open(file_path, 'r') as file:
    data_json = json.load(file)
data = data_json[0]['data']
print(data)


['人工知能は未来の仕事にどのような影響を与えるのでしょうか？', '気候変動は地球上の生態系に深刻な影響を及ぼしています。', '宇宙旅行の商業化が進むにつれ、新しい技術革新が求められています。', '健康的な食生活は長寿の秘訣とされていますが、その真実は何でしょうか？', '自動運転車の実用化は交通システムにどのような変化をもたらすでしょうか？', 'スマートシティの構想は、都市の持続可能性をどのように向上させるのでしょうか？', '古代文明の遺跡は、過去の生活様式を理解する鍵を握っています。', 'ディープラーニングは画像認識技術の進化に大きく貢献しています。', '経済格差は世界中で深刻な社会問題となっています。', '持続可能なエネルギー源への移行は、地球温暖化対策の中心的な課題です。']


In [9]:
vector_data = embed_text(data).data
print(vector_data.inputs[0])
print(vector_data.embeddings[0])


人工知能は未来の仕事にどのような影響を与えるのでしょうか？
[-0.007774353, 0.036499023, -0.00038909912, -0.010879517, -0.025985718, -0.033721924, -0.022842407, -0.040985107, 0.0042877197, 0.0024147034, 0.015808105, 0.032989502, 0.018157959, 0.004623413, -0.0016326904, 0.02999878, 0.007167816, 0.024169922, 0.051879883, 0.02507019, 0.013061523, 0.004722595, 0.0010986328, -0.029724121, -0.052093506, 0.06762695, 0.021881104, -0.05895996, 0.023162842, -0.032409668, -0.019119263, -0.0053367615, 0.033203125, 0.025268555, 0.0010757446, -0.011779785, 0.010398865, -0.027069092, 0.00065612793, 0.03756714, -0.0121154785, -0.009552002, -0.041534424, 0.005142212, -0.054138184, -0.010421753, 0.037994385, 0.03564453, -0.027359009, 0.015945435, -0.009841919, 0.0262146, 0.0093307495, -0.023895264, 0.030960083, 0.035217285, -0.027145386, 0.032226562, 0.04446411, 0.040374756, 0.036315918, 0.024612427, -0.028167725, 0.028442383, -0.038238525, 0.030029297, 0.011100769, 0.060150146, 0.011375427, 0.020507812, 0.02822876, 0.06262207, 0.023

In [10]:
# upsert data
for i in range(len(vector_data.inputs)):
    index.upsert(
        vectors=[
            {"id":"vec" + str(i), 
            "values": vector_data.embeddings[i], 
            "metadata": {"text": vector_data.inputs[i]}
            }],
        namespace=namespace
        )


In [11]:
prompt = ["機械学習"]
embed_response = embed_text(prompt)
print(embed_response.data.embeddings[0])


[-0.007896423, 0.034332275, -0.019927979, -0.014335632, -0.0033359528, -0.033111572, -0.002954483, -0.047454834, 0.011375427, -0.03604126, -0.026535034, -0.0076026917, 0.0025634766, 0.001121521, -0.049102783, 0.019607544, 0.03427124, 0.01776123, 0.058044434, 0.007549286, -0.0051956177, -0.018447876, 0.021331787, -0.06854248, -0.037750244, 0.04434204, 0.0056877136, -0.020706177, 0.004119873, -0.046020508, -0.02532959, 0.0046195984, 0.01411438, 0.025985718, -0.023849487, -0.010658264, -0.002500534, -0.059814453, 0.007843018, 0.033111572, -0.00062799454, 0.013328552, -0.033966064, 0.050598145, -0.03479004, 0.0044059753, 0.023635864, 0.056732178, -0.014549255, 0.0038547516, 0.015525818, -0.015701294, 0.0016059875, 0.0040740967, 0.042053223, 0.020126343, -0.046936035, 0.025970459, 0.050933838, 0.011962891, 0.037475586, 0.0496521, -0.011192322, 0.041534424, -0.00087070465, 0.066345215, 0.043945312, 0.05215454, 0.012298584, 0.0143966675, 0.031677246, 0.057556152, 0.017105103, -0.0061035156, -

In [12]:
response = index.query(
    vector=embed_response.data.embeddings[0], 
    top_k=3, 
    include_metadata=True, 
    # include_values=True,
    namespace=namespace
    )
print(response)


{'matches': [{'id': 'vec0',
              'metadata': {'text': '人工知能は未来の仕事にどのような影響を与えるのでしょうか？'},
              'score': 0.748510122,
              'values': []},
             {'id': 'vec7',
              'metadata': {'text': 'ディープラーニングは画像認識技術の進化に大きく貢献しています。'},
              'score': 0.674007297,
              'values': []},
             {'id': 'vec4',
              'metadata': {'text': '自動運転車の実用化は交通システムにどのような変化をもたらすでしょうか？'},
              'score': 0.62136364,
              'values': []}],
 'namespace': 'sample-namespace',
 'usage': {'read_units': 6}}


In [13]:
# langchain
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# embedding llm
embeddings = OCIGenAIEmbeddings(
    model_id=EMBED_MODEL_OCID,
    service_endpoint=GEN_AI_INFERENCE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    auth_profile=CONFIG_PROFILE,
)

# vectorstore
vectorstore = PineconeVectorStore(
    index_name=index_name, 
    embedding=embeddings, 
    namespace=namespace,
    )

# vectorsearch
query = "機械学習"
vector_search = vectorstore.similarity_search(
    query=query, 
    k=3
    )


for i in range(len(vector_search)):
    print(vector_search[i].page_content)


人工知能は未来の仕事にどのような影響を与えるのでしょうか？
ディープラーニングは画像認識技術の進化に大きく貢献しています。
自動運転車の実用化は交通システムにどのような変化をもたらすでしょうか？
